# 🔌CONEXÃO
Estabelecer conexão com os bancos

In [1]:
import psycopg2
from psycopg2.extras import DictCursor
import fdb
from dotenv import load_dotenv
import os

load_dotenv()

# Conexões
cnx_dest = fdb.connect(
    user=os.getenv("FDB_USER"),
    password=os.getenv("FDB_PASS"),
    host=os.getenv("FDB_HOST"),
    port=int(os.getenv("FDB_PORT")),
    database=os.getenv("FDB_PATH"),
    charset="WIN1252"
)
cur_dest = cnx_dest.cursor()


cnx_orig = psycopg2.connect(
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASS"),
    host=os.getenv("PG_HOST"),
    database=os.getenv("PG_DB"),
    options="-c search_path={}".format(os.getenv("PG_SCHEMA"))
)
cnx_orig.autocommit = True
cur_orig = cnx_orig.cursor(cursor_factory=DictCursor)

def commit():
    cnx_dest.commit()

# 🛠️ FERRAMENTAS
Funções, variáveis cache, hashmaps

In [2]:
global cadest, empresa
cadest = {}
empresa = cur_dest.execute("SELECT empresa FROM cadcli").fetchone()[0]

def limpa_tabela(tabelas):
    for tabela in tabelas:
        cur_dest.execute(f"DELETE FROM {tabela}")
    commit()

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

dict_modalidades = {
    "CO":{"Licit": "MAT / SERV - CONCORRENCIA", 
          "Modlic": "CON4",
          "Codmod": 4},
    "TP":{"Licit": "MAT / SERV - TOMADA", 
          "Modlic": "TOM3",
          "Codmod": 3},
    "CC":{"Licit": "MAT / SERV - CONVITE", 
          "Modlic": "CS01",
          "Codmod": 7},
    "LE":{"Licit": "LEILÃO", 
          "Modlic": "LEIL",
          "Codmod": 6},
    "DL":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "IL":{"Licit": "INEXIGIBILIDADE", 
          "Modlic": "IN01",
          "Codmod": 5},
    "PR":{"Licit": "PREGÃO PRESENCIAL", 
          "Modlic": "PP01",
          "Codmod": 8},
    "01":{"Licit": "PREGÃO PRESENCIAL", 
          "Modlic": "PP01",
          "Codmod": 8},
    "02":{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
    "03":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "04":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "06":{"Licit": "DISPENSA", 
          "Modlic": "DI01",
          "Codmod": 1},
    "CE":{"Licit": "CONCORRÊNCIA ELETRÔNICA", 
          "Modlic": "CE01",
          "Codmod": 13},
    "IN":{"Licit": "INEXIGIBILIDADE", 
          "Modlic": "IN01",
          "Codmod": 5},
    "PE":{"Licit": "PREGÃO ELETRÔNICO", 
          "Modlic": "PE01",
          "Codmod": 9},
}

def cria_coluna(tabela, coluna):
    try:
        cur_dest.execute(f"ALTER TABLE {tabela} ADD {coluna} VARCHAR(255)")
    except fdb.DatabaseError as e:
        print(f"Erro ao criar coluna {coluna} na tabela {tabela}: {e}")
    else:
        commit()

# 🛒 COMPRAS
<p>Extração, tratamento e carregamento dos dados referentes ao módulo compras</p>

## CADASTROS BASE

### CADUNIMEDIDA

In [3]:
limpa_tabela(("cadunimedida",))

insert = cur_dest.prep("insert into cadunimedida(sigla,descricao) values(?,?)")

cur_orig.execute("select distinct substring(sigla,0,4) sigla, descricao from cadunidademedida where trim(sigla) <> ''")

for row in cur_orig:
    cur_dest.execute(insert, (row[0], row[1]))
commit()


### GRUPO E SUBGRUPO

In [4]:
limpa_tabela(("cadsubgr", "cadgrupo"))

cur_orig.execute("""
select
	distinct
	to_char(grupo::integer, 'fm000') grupo,
	subgrupo,
	a.descricao
from
	cgasubgrupos a
union all
select distinct
	to_char(grupo::integer, 'fm000') grupo,
	concat(substring(mercadoria,1,1), substring(subgrupo,2,2)) subgrupo,
	b.descricao
	--, a.mercadoria
from
	cgc_vw_mercadorias a
left join cgasubgrupos b using (grupo, subgrupo)
where a.mercadoria not like '0%'                  
union all
	select
	distinct
	to_char(grupo::integer, 'fm000') grupo,
	'000',
	a.descricao
from
	cgagrupos a
order by
	2,
	1
""")

for row in cur_orig.fetchall():
	try:
		if row['subgrupo'] == '000':
			cur_dest.execute("insert into cadgrupo(grupo, nome) values(?, ?)", (row['grupo'], row['descricao'][:45]))
		else:
			cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values(?, ?, ?)", (row['grupo'], row['subgrupo'], row['descricao'][:45]))
		commit()
	except Exception as e:
		print(f"Erro ao inserir grupo/subgrupo {row['grupo']}/{row['subgrupo']}: {e} - {row.items()}")

cur_dest.execute("insert into cadgrupo(grupo, nome) values ('700', 'CONSUMO')")
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('800', 'BEM')")
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('900', 'SERVIÇOS')")
cur_dest.execute("insert into cadgrupo(grupo, nome) values ('000', 'SEM IDENTIFICAÇÃO')")
commit()

cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('700', '000', 'CONSUMO')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('800', '000', 'BEM')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('900', '000', 'SERVIÇOS')")
cur_dest.execute("insert into cadsubgr(grupo, subgrupo, nome) values ('000', '000', 'SEM IDENTIFICAÇÃO')")
commit()

### CADEST

In [5]:
limpa_tabela(("cadest",))
cria_coluna("cadest", "key")

insert = cur_dest.prep("insert into cadest(grupo, subgrupo, codigo, cadpro, codreduz, disc1, ocultar, unid1, tipopro, usopro, key) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)")

cur_orig.execute("""
with cte as (
select
	case 
		when tipo = 1 and nullif(trim(grupo), '') is null then '700'
		when tipo = 2 and nullif(trim(grupo), '') is null then '800'
		when tipo = 3 and nullif(trim(grupo), '') is null then '900'
		else to_char(nullif(trim(grupo), '')::int, 'fm000')
	end grupo,
	format(coalesce(nullif(trim(subgrupo), ''), '000'), 'fm000') subgrupo,
	trim(descricao) disc1,
	abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
	cvi.unidemb unid1,
	tipo
from
	cgc_vw_itensprocesso cvi)
select distinct
	grupo,
	subgrupo,
	codreduz,
	disc1,
	unid1,
	tipo
from
	cte
order by 1, 2, 3
""")

subgrupos_desdobrados = {k:[v, 0] for k, v in cur_dest.execute("select grupo||'.'||subgrupo, cast(subgrupo as integer) subgrupo_int from cadsubgr")}
max_subgrupo_grupo = {k: v for k, v in cur_dest.execute("select grupo, max(cast(subgrupo as integer)) from cadsubgr group by 1")}

tipo_uso_produto = {
	1: ['P', 'C'],
	2: ['P', 'P'],
	3: ['S', '']
}

for row in cur_orig.fetchall():
	key = f"{row['grupo']}.{row['subgrupo']}"
	subgrupo_int, codigo = subgrupos_desdobrados[key]
	codigo += 1

	if codigo > 999:
		subgrupo_ant = max_subgrupo_grupo[row['grupo']]
		subgrupo_int = max_subgrupo_grupo[row['grupo']] + 1
		codigo = 1
		subgrupos_desdobrados[key] = [subgrupo_int, codigo]
		max_subgrupo_grupo[row['grupo']] = subgrupo_int
	else:
		subgrupos_desdobrados[key][1] = codigo

	tipopro, usopro = tipo_uso_produto.get(row['tipo'], ['P', 'C'])

	cadpro = f'{row['grupo']}.{subgrupo_int:03}.{codigo:03}'

	cur_dest.execute(insert, (row['grupo'], f'{subgrupo_int:03}', codigo, cadpro, row['codreduz'], row['disc1'][:1024], 'N', row['unid1'], tipopro, usopro, key))
commit()

cur_dest.execute("""
INSERT INTO CADSUBGR (grupo, subgrupo, nome)
WITH CTE AS (
SELECT DISTINCT a.grupo, a.subgrupo, KEY FROM cadest a
WHERE NOT EXISTS (SELECT 1 FROM cadsubgr x WHERE a.grupo = x.grupo AND a.SUBGRUPO = x.SUBGRUPO))
SELECT a.GRUPO, a.SUBGRUPO, b.NOME FROM cte a
JOIN CADSUBGR b ON b.grupo||'.'||b.SUBGRUPO = a."KEY" 
""")
commit()

cadest = {k: v for k,v in cur_dest.execute("select codreduz, max(cadpro) from cadest group by codreduz").fetchall()}

### ALMOXARIFADOS

In [6]:
limpa_tabela(("destino",))
cria_coluna("destino", "almox_pai")

insert = cur_dest.prep("""insert into destino(cod,desti,empresa,almox_pai) values(?,?,?,?)""")

cur_orig.execute("""
select
	to_char(id, 'fm000000000') cod,
	nome desti,
	g.unidadeprincipal pai,
    localfisico codccusto
from
	gapunidadealmox g  
""")

for row in cur_orig.fetchall():
    cur_dest.execute(insert, (row['cod'], row['desti'], empresa, row['pai']))
commit()

### CENTRO DE CUSTO

In [7]:
limpa_tabela(("centrocusto",))

try:
    cur_dest.execute(f"insert into destino(cod,desti,empresa) values('000000000','CONVERSAO',{empresa})")
except fdb.DatabaseError as e:
    print(f"Erro ao inserir destino: {e}")
else:
    commit()

insert = cur_dest.prep("""
    insert into centrocusto (codccusto, descr, ccusto, ocultar, empresa, poder, orgao, destino, pai) values (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
with CTE as (
    select
        id as codccusto,
        nome as descr,
        pai,
        versao,
        row_number() over (partition by id order by versao desc) as rn
    from
        pinhalpm.cadlocalfisico
)
select
    codccusto,
    descr,
    pai
from
    CTE a
where
    rn = 1;
""")

for row in cur_orig:
    cur_dest.execute(insert, (row['codccusto'], row['descr'], '001', 'N', empresa, '02', '01', '000000000', row['pai']))
commit()

DatabaseError: ('Error while preparing SQL statement:\n- SQLCODE: -206\n- Dynamic SQL Error\n- SQL error code = -206\n- Column unknown\n- PAI\n- At line 2, column 97', -206, 335544569)



## COTAÇÃO



### CADORC

In [ ]:
limpa_tabela(("cadorc",))

insert = cur_dest.prep("""
insert into cadorc
	(id_cadorc,
	num,
	ano,
	numorc,
	dtorc,
	descr,
	prioridade,
	obs,
	status,
	liberado,
	codccusto,
	liberado_tela,
	empresa) values
	(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
    concat(c.processo, EXTRACT(year from c.ANO)::INT%2000) id_cadorc,
    to_char(c.processo, 'fm00000') num,
    EXTRACT(year from c.ANO)::INT ano,
    concat(to_char(c.processo, 'fm00000/'),EXTRACT(year from c.ANO)::INT%2000) numorc,
    c.data dtorc,
    CONCAT(
        'SOLICITACOES: ',
        string_agg(
            distinct concat(to_char(a.solicitacao, 'fm00000/'),
                            EXTRACT(year from a.ANO)::INT%2000),
            ', ' order by concat(to_char(a.solicitacao, 'fm00000/'),
                                 EXTRACT(year from a.ANO)::INT%2000)
        )
    ) as obs,
    (array_agg(a.destinacao order by a.solicitacao))[1] as descr,
    case
        when c.status = 'A' then 'CA'
        when c.status = 'I' then 'AB'	
        else 'EC'
    end status,
    case when c.status = 'C' then 'S' else 'N' end liberado,
    0 codccusto,
    case when c.status = 'C' then 'L' end liberado_tela
from pinhalpm.cgcsolicitacoes a
join (
    select distinct processo, ano, anosolicitacao, solicitacao
    from pinhalpm.cgc_vw_itenssolxprcp
) b 
    on a.ano = b.anosolicitacao
    and a.solicitacao = b.solicitacao
join pinhalpm.cgcprocessocompra c 
    on b.ano = c.ano
    and b.processo = c.processo
group by c.processo, c.ano, c.data, c.status
order by c.ano, c.processo
""")

for row in cur_orig:
	cur_dest.execute(insert, (
		row['id_cadorc'],
		row['num'],
		row['ano'],
		row['numorc'],
		row['dtorc'],
		row['descr'][:1024],
		'NORMAL',
		row['obs'],
		row['status'],
		row['liberado'],
		row['codccusto'],
		row['liberado_tela'],
		empresa
	))
commit()

### ICADORC

In [9]:
limpa_tabela(("icadorc",))

insert = cur_dest.prep("""
insert into icadorc (
    numorc,
    item,
    cadpro,
    qtd, 
    valor, 
    itemorc, 
    codccusto, 
    itemorc_ag,
    id_cadorc)
values(?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute("""with CTE as (
select
	concat(to_char(processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
    row_number() over (partition by processo, a.ano order by a.tipo, processo, a.ano, a.item) item,
    abs(hashtext(concat(trim(descricao),unidemb))) codreduz,
    quantidade,
    0 valorestimado,
    concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc,
    0 codccusto
from
	cgc_vw_itensprocesso a)
select * from cte a""")

for row in cur_orig:
    try:
        key = str(row['codreduz'])
        cadpro = cadest[key]
    except Exception as e:
        print(f"Erro ao obter cadpro: {e}")
        break

    else:
        cur_dest.execute(insert, (
            row['numorc'],
            row['item'],
            cadpro,
            row['quantidade'],
            row['valorestimado'],
            row['item'],
            0,
            row['item'],
            row['id_cadorc']
        ))
commit()

cur_dest.execute("update icadorc a set a.codccusto = (select codccusto from cadorc b where a.id_cadorc = b.id_cadorc)")
commit()
    

### FCADORC

In [10]:
limpa_tabela(("fcadorc",))

insert = cur_dest.prep("insert into fcadorc(numorc, codif, nome, valorc, id_cadorc) values (?,?,?,?,?)")

cur_orig.execute("""
select
	concat(to_char(a.processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
	coalesce(fornecedor, abs(hashtext(concat(trim(fornaocadastrado))))) codif,
	coalesce(c.nome, fornaocadastrado) nome,
	coalesce(sum(valor*d.quantidade),0) valorc,
	concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc
from
	cgcprevia a
join cgcpreviaitens b using (processo, ano, previa)
left join cgcfornecedor c using (fornecedor)
left join cgc_vw_itensprocesso d using (tipo, processo, ano, item)
group by 1,2,3,5
""")

for row in cur_orig:
	cur_dest.execute(insert, (
		row['numorc'],
		row['codif'],
		row['nome'][:70],
		row['valorc'],
		row['id_cadorc']
	))
commit()

### VCADORC

In [ ]:
limpa_tabela(("vcadorc",))

insert = cur_dest.prep("""insert into vcadorc(numorc, item, codif, vlruni, vlrtot, ganhou, vlrganhou, classe, id_cadorc) values (?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
select
    concat(to_char(a.processo, 'fm00000/'),EXTRACT(year from a.ANO) :: INT%2000) numorc,
    coalesce(fornecedor, abs(hashtext(concat(trim(fornaocadastrado))))) codif,
    dense_rank() over (
        partition by processo, a.ano
        order by a.tipo, a.item
    ) as item,
    valor vlruni,
    valor*quantidade vlrtot,
    'UN' classe,
    concat(a.processo, EXTRACT(year from a.ANO) :: INT%2000) id_cadorc
from
	cgcpreviaitens a
join cgcprevia b using (processo, ano, previa)
join (select processo, ano, tipo, item, quantidade from cgc_vw_itensprocesso) c using (processo, ano, tipo, item)
where valor is not null
""")

for row in cur_orig:
    cur_dest.execute(insert, (
        row['numorc'],
        row['item'],
        row['codif'],
        row['vlruni'],
        row['vlrtot'],
        None,
        None,
        'UN',
        row['id_cadorc']
    ))
commit()

cur_dest.execute("""merge into vcadorc a
using (
    SELECT 
        id_cadorc, 
        item, 
        codif, 
        vlruni, 
        ROW_NUMBER() OVER (
            PARTITION BY id_cadorc, item 
            ORDER BY vlruni ASC, codif ASC
        ) AS rn
    FROM vcadorc
) b
on a.id_cadorc = b.id_cadorc
and a.item = b.item
and b.rn = 1
when matched then
    update set 
        a.vlrganhou = b.vlruni,
        a.ganhou = b.codif""")
commit()

## LICITAÇÕES

### CADLIC

In [ ]:
limpa_tabela(("cadlic"))

insert = cur_dest.prep("""
INSERT
		INTO
		cadlic(
		numlic,
		proclic,
		numero,
		ano,
		comp,
		licnova,
		liberacompra,
		discr,
		detalhe,
		registropreco,
		microempresa,
		numpro,
		discr7,
		datae,
		processo_data,
		dtadj,
		dthom,
		codtce,
		anomod,
		modlic,
		licit,
		codmod,
		dtpub,
		dtenc,
		valor,
		empresa,
		processo,
		processo_ano,
		dtreal)
	VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
	?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
	concat(numero,EXTRACT(YEAR FROM a.ano)::INT%2000) numlic,
	concat(to_char(numero,'fm000000'),'/',EXTRACT(YEAR FROM a.ano)::INT%2000) proclic,
	to_char(numero,'fm000000') numero,
	EXTRACT(YEAR FROM a.ano)::INT ano,
	case when percaprovado is not null then 3 else 1 end comp,
	1 licnova,
	'S' liberacompra,
	objeto discr,
	objetochar detalhe,
    case when upper(objeto) like '%REGISTRO DE PREÇO%' then 'S' else 'N' end registropreco,
	'3' microempresa,
	nromodalidade numpro,
	'Menor Preco Unitario' discr7,
	digitacao datae,
	anoproc processo_data,
	datahomologacao dthom,
	b.codigoaudesp codtce,
	publicacao dtpub,
	valorestimado,
	processocompra,
	anoproc,
	coalesce(modalidade,'DL') modlic
from
	liclicitacao a
left join licaudesp b on a.numero = b.licitacao and a.ano = b.ano and b.tipoprocesso = 1
""")

for row in cur_orig:
    info_modalidades = dict_modalidades.get(row.numpro, {"Licit": "DISPENSA", "Modlic": "DI01", "Codmod": 1})

    licit = info_modalidades["Licit"]
    modlic = info_modalidades["Modlic"]
    codmod = info_modalidades["Codmod"]

    cur_dest.execute(insert, (
        row['numlic'], 
        row['proclic'],
        row['numero'],
        row['ano'],
        row['comp'],
        row['licnova'],
        row['liberacompra'],
        row['discr'],
        row['detalhe'],
        row['registropreco'],
        row['microempresa'],
        row['numpro'],
        row['discr7'],
        row['datae'],
        row['processo_data'],
        row['dthom'],
        row['dthom'],
        row['codtce'],
        row['ano'],
        modlic,
        licit,
        codmod,
        row['dtpub'],
        row['dthom'],
        row['valorestimado'],
        empresa,
        row['processocompra'],
        row['anoproc'],
        row['datae']))
commit()

cur_dest.Exec("""EXECUTE BLOCK AS
DECLARE VARIABLE DESCMOD VARCHAR(1024);
DECLARE VARIABLE CODMOD INTEGER;
BEGIN
    FOR
        SELECT CODMOD, DESCMOD FROM MODLIC INTO :CODMOD, :DESCMOD
    DO
    BEGIN
        UPDATE CADLIC SET LICIT = :DESCMOD where CODMOD = :CODMOD;
    END
END""")

cur_dest.Exec("""INSERT
    INTO
    MODLICANO (ULTNUMPRO,
    CODMOD,
    ANOMOD,
    EMPRESA)
SELECT
    COALESCE(MAX(NUMPRO), 0),
    CODMOD,
    COALESCE(ANO, 0) ANO,
    EMPRESA
FROM
    CADLIC c
WHERE
    CODMOD IS NOT NULL
GROUP BY
    2,
    3,
    4
ORDER BY
    ano,
    codmod""")

cur_dest.Exec("""UPDATE CADLIC SET FK_MODLICANO = (SELECT PK_MODLICANO FROM MODLICANO WHERE CODMOD = CADLIC.CODMOD AND ANOMOD = CADLIC.ANO AND CADLIC.EMPRESA = MODLICANO.EMPRESA) WHERE CODMOD IS NOT NULL""")
commit()

### CADLIC SESSAO

In [ ]:
cur_dest.Exec("""INSERT INTO CADLIC_SESSAO (NUMLIC, SESSAO, DTREAL, HORREAL, COMP, DTENC, HORENC, SESSAOPARA, MOTIVO) 
SELECT L.NUMLIC, CAST(1 AS INTEGER), L.DTREAL, L.HORREAL, L.COMP, L.DTENC, L.HORENC, CAST('T' AS VARCHAR(1)), CAST('O' AS VARCHAR(1)) FROM CADLIC L 
WHERE numlic not in (SELECT FIRST 1 S.NUMLIC FROM CADLIC_SESSAO S WHERE S.NUMLIC = L.NUMLIC)""")
commit()

### PARTICIPANTES

In [ ]:
limpa_tabela(("prolics","prolic"))

insert_prolic = cur_dest.prep("insert into prolic(numlic,codif,nome,status) values(?,?,?,?)")
insert_prolics = cur_dest.prep("insert into prolics(sessao,numlic,codif,habilitado,status,cpf,representante) values(?,?,?,?,?,?,?)")

cur_orig.execute("""
select
	concat(licitacao,EXTRACT(YEAR FROM ano)::INT%2000) numlic,
	fornecedor,
	nome,
    cgccpf
from
	liclicfornecedor a
left join cgcfornecedor b using (fornecedor)
""")

for row in cur_orig:
    cur_dest.execute(insert_prolic, (row.numlic, row.fornecedor, row.nome, 'A'))
    cur_dest.execute(insert_prolics, (1, row.numlic, row.fornecedor, 'S', 'A', row.cgccpf, row.fornecedor))
commit()

### CADPROLIC

In [ ]:
limpa_tabela(("cadprolic_detalhe_fic", "cadprolic_detalhe", "cadprolic"))

insert_cadprolic = cur_dest.prep("""
INSERT INTO
		cadprolic(item,
		item_mask,
		cadpro,
		quan1,
		vamed1,
		vatomed1,
		codccusto,
		reduz,
		numlic)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

insert_cadprolic_detalhe = cur_dest.prep("""insert into cadprolic_detalhe (numlic, item, cadpro, quan1, vamed1, vatomed1, codccusto) values (?, ?, ?, ?, ?, ?, ?) """)

insert_cadprolic_detalhe_fic = cur_dest.prep("""
insert into 
        cadprolic_detalhe_fic (numlic, item, codigo, qtd, valor, qtdadt, valoradt, codccusto, qtdmed, valormed, tipo) 
values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

cur_orig.execute("""
select
	item,
	coalesce(idespecificacao,0) codreduz,
	a.qtde quan1,
	coalesce(a.valor,0) vaun1,
	coalesce(qtde*valor,0) vato1,
	0 codccusto,
	concat(licitacao, extract(year from ano)::INT%2000) numlic
from
	liclicitens a
left join cgc_vw_mercadorias b on 
	concat(grupo,'.',subgrupo,'.',mercadoria) = a.codigoitem
order by numlic, item
""")

for row in cur_orig:
    try:
        cadpro = cadest[row.codigoitem]
    except KeyError:
        print(f"Item {row.codigoitem} não encontrado em cadest.")
        break

    cur_dest.execute(insert_cadprolic, (row.item, row.item, cadpro, row.quan1, row.vaun1, row.vato1, row.codccusto, 'N', row.numlic))
    cur_dest.execute(insert_cadprolic_detalhe, (row.numlic, row.item, cadpro, row.quan1, row.vaun1, row.vato1, row.codccusto))
    cur_dest.execute(insert_cadprolic_detalhe_fic, (row.numlic, row.item, row.item, row.quan1, row.vato1, row.quan1, row.vato1, row.codccusto, row.quan1, row.vato1, 'C'))
commit()

### PROPOSTA

In [ ]:
limpa_tabela(("cadpro", "cadpro_final", "cadpro_proposta"))

insert = cur_dest.prep("""
INSERT
    INTO
    cadpro_proposta(sessao,
    codif,
    item,
    itemp,
    quan1,
    vaun1,
    vato1,
    numlic,
    status,
    subem,
    marca,
    itemlance)
VALUES(?,?,?,?,?,?,?,?,?,?,?,?)""")

cur_orig.execute("""
select
	fornecedor codif,
	itempr,
	qtde_total quan1,
	valor vaun1,
	qtde_total*valor vato1,
	concat(licitacao,EXTRACT(YEAR FROM ano)::INT%2000) numlic,
	'C' status,
	'S' itemlance
from
	lic_vw_resultado_prcompra
""")           

for row in cur_orig:
    cur_dest.execute(insert, (1, row.codif, row.itempr, row.itempr, row.quan1, row.vaun1, row.vato1, row.numlic, row.status, 1, None, row.itemlance, 'N'))
commit()

### CADPRO LANCE

In [ ]:
cur_dest.execute("""insert into cadpro_lance (sessao, rodada, codif, itemp, vaunl, vatol, status, subem, numlic)
	SELECT sessao, 1 rodada, CODIF, ITEMP, VAUN1, VATO1, 'F' status, SUBEM, numlic FROM CADPRO_PROPOSTA cp where subem = 1 and not exists
	(select 1 from cadpro_lance cl where cp.codif = cl.codif and cl.itemp = cp.itemp and cl.numlic = cp.numlic)""")
commit()

### CADPRO FINAL

In [ ]:
cur_dest.execute("""INSERT into cadpro_final (numlic, ult_sessao, codif, itemp, vaunf, vatof, STATUS, subem)
	SELECT numlic, sessao, codif, itemp, vaun1, vato1, CASE WHEN status = 'F' THEN 'C' ELSE status end, subem FROM cadpro_proposta
	WHERE NOT EXISTS (SELECT 1 FROM cadpro_final f WHERE f.numlic = cadpro_proposta.numlic AND f.itemp = cadpro_proposta.itemp AND f.codif = cadpro_proposta.codif)""")
commit()

### CADPRO

In [ ]:
cur_dest.execute("""INSERT INTO CADPRO (
    CODIF,
    CADPRO,
    QUAN1,
    VAUN1,
    VATO1,
    SUBEM,
    STATUS,
    ITEM,
    NUMORC,
    ITEMORCPED,
    CODCCUSTO,
    FICHA,
    ELEMENTO,
    DESDOBRO,
    NUMLIC,
    ULT_SESSAO,
    ITEMP,
    QTDADT,
    QTDPED,
    VAUNADT,
    VATOADT,
    PERC,
    QTDSOL,
    ID_CADORC,
    VATOPED,
    VATOSOL,
    TPCONTROLE_SALDO,
    QTDPED_FORNECEDOR_ANT,
    VATOPED_FORNECEDOR_ANT,
    marca
)
SELECT
    a.CODIF,
    c.CADPRO,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) ELSE 0 END qtdunit,
    a.VAUNL,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) * a.VAUNL ELSE 0 END VATOTAL,
    1,
    'C',
    c.ITEM,
    c.NUMORC,
    c.ITEM,
    c.CODCCUSTO,
    c.FICHA,
    c.ELEMENTO,
    c.DESDOBRO,
    a.NUMLIC,
    1,
    b.ITEMP,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) ELSE 0 END qtdunit,
    0,
    a.VAUNL,
    CASE WHEN a.VAUNL <> 0 THEN ROUND((a.vatol / a.VAUNL), 2) * a.VAUNL ELSE 0 END vatoadt,
    0,
    0,
    c.ID_CADORC,
    0,
    0,
    'Q',
    0,
    0,
    p.marca
FROM
    CADPRO_LANCE a
INNER JOIN CADPRO_STATUS b ON
    b.NUMLIC = a.NUMLIC AND a.ITEMP = b.ITEMP AND a.SESSAO = b.SESSAO
INNER JOIN CADPROLIC_DETALHE c ON
    c.NUMLIC = a.NUMLIC AND b.ITEMP = c.ITEM_CADPROLIC
INNER JOIN CADLIC D ON
    D.NUMLIC = A.NUMLIC
inner join cadpro_proposta p on 
    p.numlic = a.numlic and p.itemp = a.itemp and p.codif = a.codif
WHERE
    a.SUBEM = 1 AND a.STATUS = 'F'
    AND NOT EXISTS (
        SELECT 1 
        FROM CADPRO cp
        WHERE cp.NUMLIC = a.NUMLIC 
        AND cp.ITEM = c.ITEM 
        AND cp.CODIF = a.CODIF
    )""")
commit()

### REGPRECO

In [ ]:
cur_dest.execute("""
EXECUTE BLOCK AS  
    BEGIN  
    INSERT INTO REGPRECODOC (NUMLIC, CODATUALIZACAO, DTPRAZO, ULTIMA)  
    SELECT DISTINCT A.NUMLIC, 0, DATEADD(1 YEAR TO A.DTHOM), 'S'  
    FROM CADLIC A WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECODOC X  
    WHERE X.NUMLIC = A.NUMLIC);  

    INSERT INTO REGPRECO (COD, DTPRAZO, NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, QTDENT, SUBEM, STATUS, ULTIMA)  
    SELECT B.ITEM, DATEADD(1 YEAR TO A.DTHOM), B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, 0, B.SUBEM, B.STATUS, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECO X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  

    INSERT INTO REGPRECOHIS (NUMLIC, CODIF, CADPRO, CODCCUSTO, ITEM, CODATUALIZACAO, QUAN1, VAUN1, VATO1, SUBEM, STATUS, MOTIVO, MARCA, NUMORC, ULTIMA)  
    SELECT B.NUMLIC, B.CODIF, B.CADPRO, B.CODCCUSTO, B.ITEM, 0, B.QUAN1, B.VAUN1, B.VATO1, B.SUBEM, B.STATUS, B.MOTIVO, B.MARCA, B.NUMORC, 'S'  
    FROM CADLIC A INNER JOIN CADPRO B ON (A.NUMLIC = B.NUMLIC) WHERE A.REGISTROPRECO = 'S' AND A.DTHOM IS NOT NULL  
    AND NOT EXISTS(SELECT 1 FROM REGPRECOHIS X  
    WHERE X.NUMLIC = B.NUMLIC AND X.CODIF = B.CODIF AND X.CADPRO = B.CADPRO AND X.CODCCUSTO = B.CODCCUSTO AND X.ITEM = B.ITEM);  
END;""")
commit()

# 📦 ALMOXARIFADO
Extração, tratamento e carregamentos dos dados referentes ao módulo de almoxarifado

In [ ]:
limpa_tabela(("icadreq", "requi"))

insert_requi = cur_dest.prep("""
INSERT
    INTO
    requi (empresa,
    id_requi,
    requi,
    num,
    ano,
    destino,
    codccusto,
    datae,
    dtlan,
    dtpag,
    entr,
    said,
    comp,
    codif,
    entr_said)
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

insert_icadreq = cur_dest.prep("""
    insert into icadreq (id_requi, requi, codccusto, empresa, item, quan1, quan2, vaun1, vaun2, vato1, vato2, cadpro, destino) values (?,?,?,?,?,?,?,?,?,?,?,?,?)
""")



# 🏛️ PATRIMÔNIO
Extração, tratamento e carregamento dos dados referentes ao módulo patrimônio

## CADASTROS BASE

### TIPO DE MOVIMENTAÇÕES

In [ ]:
limpa_tabela(("pt_tipomov"))

valores = {
    "A": "Aquisição",
    "B": "Baixa",
    "T": "Transferência",
    "R": "Procedimento Contábil",
    "P": "Transferência de Plano Contábil",
}

cur_dest.executemany("INSERT INTO PT_TIPOMOV (codigo_tmv, descricao_tmv) VALUES (?, ?)", valores.items())

### TIPO DE AJUSTE

In [ ]:
limpa_tabela(("pt_cadbai"))

cur_dest.execute("INSERT INTO PT_CADAJUSTE (CODIGO_AJU, EMPRESA_AJU, DESCRICAO_AJU) VALUES (1, ?, 'REAVALIAÇÃO (ANTES DO CORTE)')", (empresa))

### TIPOS DE BAIXA

In [ ]:
limpa_tabela(("pt_cadbai"))

insert = cur_dest.prep("INSERT INTO PT_CADBAI (CODIGO_BAI, EMPRESA_BAI, DESCRICAO_BAI) VALUES (?, ?, ?)")

cur_orig.execute("select id, descricao from pinhalpm.sppmotivobaixa s")

for row in cur_orig:
    cur_dest.execute(insert, (row.id, empresa, row.descricao))
commit()

### CONSERVAÇÃO

In [ ]:
limpa_tabela(("pt_cadsit"))

insert = cur_dest.prep("INSERT INTO PT_CADSIT (CODIGO_SIT, EMPRESA_SIT, DESCRICAO_SIT) VALUES (?, ?, ?)")

cur_orig.execute("select id, s.estadoconservacao  from sppconservaporcent s")

for row in cur_orig:
    cur_dest.execute(insert, (row.id, empresa, row.estadoconservacao))
commit()

### TIPOS DE BEM

In [ ]:
limpa_tabela(("pt_cadtip"))

insert = cur_dest.prep("INSERT INTO PT_CADTIP (CODIGO_TIP, EMPRESA_TIP, DESCRICAO_TIP) VALUES (?, ?, ?)")

cur_orig.execute("select id, nome from pinhalpm.gcpespecificacao as g order by 1")

for row in cur_orig:
    cur_dest.execute(insert, (row.id, empresa, row.nome))
commit()

### UNIDADE E SUBUNIDADE

In [ ]:
limpa_tabela(("pt_cadpats", "pt_cadpatd"))

cur_dest.execute("INSERT INTO PT_CADPATD (EMPRESA_DES, CODIGO_DES, NAUNI_DES, OCULTAR_DES) select ?, 1, clnt1, 'N' from cadcli", (empresa))

commit()

insert = cur_dest.prep(f"INSERT INTO PT_CADPATS (codigo_set, empresa_set, codigo_des_set, noset_set, ocultar_set) VALUES (?, {empresa}, 1, ?, 'N')")

cur_orig.execute("select id, nome from cadlocalfisico c order by 1")

for row in cur_orig:
    cur_dest.execute(insert, (row.id, row.nome))
commit()

### GRUPO

In [ ]:
limpa_tabela(("pt_cadpatg"))

cur_dest.execute(f"INSERT INTO PT_CADPATG (CODIGO_GRU, EMPRESA_GRU, NOGRU_GRU) VALUES (1, {empresa}, 'GERAL')")

commit()

## BENS

In [ ]:
limpa_tabela(("pt_cadpat"))

dict_conservacoes = {k:v for k, v in cur_dest.execute("SELECT descricao_sit, codigo_sit FROM pt_cadsit").fetchall()}
dict_codigos_cpl = {k:v for k, v in cur_dest.execute("SELECT titco, balco FROM cadest").fetchall()}

insert = cur_dest.prep(f"""
insert into pt_cadpat (
    codigo_pat,
    empresa_pat,
    codigo_gru_pat,
    chapa_pat,
    codigo_cpl_pat,
    codigo_set_pat,
    codigo_set_atu_pat,
    orig_pat,
    codigo_tip_pat,
    codigo_sit_pat,
    discr_pat,
    datae_pat,
    dtlan_pat,
    valaqu_pat,
    valatu_pat,
    codigo_for_pat,
    percenqtd_pat,
    dae_pat,
    valres_pat,
    percentemp_pat,
    nempg_pat,
    anoemp_pat,
    dtpag_pat,
    hash_sinc,
    codigo_bai_pat)
VALUES(?,{empresa},1,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""")

cur_orig.execute("""
select
	a.id codigo_pat,
	1 codigo_gru_pat,
	to_char(a.codigoreduzido, 'fm000000') chapa_pat,
	b.nome nome_cpl,
	c.localdestino codigo_set_pat,
	a.localfisicoatual codigo_set_atu_pat,
	orig_pat,
	a.especificacao codigo_tip_pat,	
	a.estadoincorporacao nome_sit,
	a.descricao discr_pat,
	a.tombamento datae_pat,
	a.inicioatividade dtlan_pat,
	a.valorincorporacao valaqu_pat,
	a.valoratual valatu_pat,
	a.fornecedor codigo_for_pat,
	a.vidautil,
	case when a.vidautil is not null then 'V' else 'N' end dae_pat,
	a.valorincorporacao*(a.percvlrresidual/100) valres_pat,
	'M' percentemp_pat,
	e.datamovimentacao dtpag_pat,
	e.motivo_baixa codigo_bai_pat,
    f.empenho,
	f.anoempenho
from
	sppbempatrimonial a
join gcpespecificacao b on a.especificacao = b.id
left join (with transferencias as (select
		x2.bempatrimonial,
		x1.localdestino,
		x2.movimentacao,
		row_number() over (partition by bempatrimonial order by movimentacao) rn
	from
		sppbemtransfere x1
	join sppmovbem x2 on
		x1.id = x2.movimentacao
	where
		localorigem is null
	order by 1, 3)
	select bempatrimonial, localdestino from transferencias where rn = 1) c on c.bempatrimonial = a.id
left join (select
		bempatrimonial,
		case when substring(tipo_incorporacao,1,1) = 'L' then 'I'
		when substring(tipo_incorporacao,1,1) = 'A' then 'C'
		else substring(tipo_incorporacao,1,1) end orig_pat
	from
		spp_vw_incorporacoes) d on d.bempatrimonial = a.id
left join (select datamovimentacao, motivo_baixa, bempatrimonial from spp_vw_baixas) e
on e.bempatrimonial = a.id
left join (select bempatrimonial, empenho, anoempenho from spp_vw_bemorigem) f on
a.id = f.bempatrimonial
""")

for row in cur_orig:
    codsit = dict_conservacoes.get(row.nome_sit, 1)
    codigo_cpl_pat = dict_codigos_cpl.get(row.codigo_pat, None)

    cur_dest.execute(insert, (
        row.codigo_pat,
        row.chapa_pat,
        codigo_cpl_pat,
        row.codigo_set_pat,
        row.codigo_set_atu_pat,
        row.orig_pat,
        row.codigo_tip_pat,
        codsit,
        row.discr_pat,
        row.datae_pat,
        row.dtlan_pat,
        row.valaqu_pat,
        row.valatu_pat,
        row.codigo_for_pat,
        row.vidautil,
        row.dae_pat,
        row.valres_pat,
        row.percentemp_pat,
        row.empenho,
        row.anoempenho,
        row.dtpag_pat,
        row.codigo_pat,
        row.codigo_bai_pat
    ))
commit()